<a href="https://colab.research.google.com/github/biondo999/Cfd/blob/main/labs/lab7/TEMPLATE_CFDlab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np

from firedrake.petsc import PETSc

In [ ]:
# Get current path: all relative paths that you may use for input/output start from here.
#
# - Colab:  The default path is /content, and your GDrive folder is accessible (if mounted) at /content/drive/MyDrive
#
# - local:  If running on your local machine, current_path is the position WHERE YOU LAUNCHED THE NOTEBOOK KERNEL BY 'jupyter notebook'
#           and NOT the directory in which this ipynb file is saved.
#           If you want to modify your current path, go to the terminal, stop the kernel (ctrl-C + press y), then move to the desired path by
#           cd my/desired/path/starting/from/here
#           and then launch 'jupyter notebook'.
import os
current_path = os.getcwd()
print(current_path)

# my_io_path = '/content/drive/MyDrive/Colab Notebooks/CFD2324/'
my_io_path = '/content/drive/MyDrive/Cfd/lab7'+'/'
print(my_io_path)

---
---
# Exercise 1

\begin{equation*}
\begin{cases}
(\boldsymbol{u}\cdot\nabla)\boldsymbol{u} - \nu\Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega, \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega, \\
\boldsymbol{u} = 4y(1-y)\boldsymbol{i} & {\rm on} \ \Gamma_{\rm in}, \\
(\nu\nabla\boldsymbol{u} - pI)\boldsymbol{n} = \boldsymbol{0} & {\rm on} \ \Gamma_{\rm out}, \\
\left[\begin{aligned}
\text{(i) }&\boldsymbol{u} = \boldsymbol{0}\\
\text{(ii) }&\boldsymbol{u}\cdot\boldsymbol{n} = 0, \quad (\nu\nabla\boldsymbol{u}-pI)\boldsymbol{n} \cdot \boldsymbol{t} = 0
\end{aligned}\right]
& {\rm on} \ \Gamma_{\rm wall}=\partial\Omega\setminus(\Gamma_{\rm in}\cup\Gamma_{\rm out}). \\
\end{cases}
\end{equation*}

In [ ]:
# Import mesh: set the path correctly!
# See cell before Exercise 1 about current path.

#mesh = Mesh(my_io_path+'meshes/elbow3bis.msh')
my_mesh='/content/drive/MyDrive/Cfd/lab7/elbow3bis.msh'
mesh=Mesh(my_mesh)

fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend(loc='upper left')

### Variational form - residual

In [ ]:
# inputs:   uh, ph  components of an actual Function (NOT trial functions)
#           v, q    components of a TestFunction
#           nu, f   viscosity and source function
# output:   G       linear form (i.e. functional) representing the evaluation of
#                   the residual for a given solution (uh,ph), for any test function
def nonlinear_problem_residual(uh, v, ph, q, nu, f):
    G= nu*inner(grad(uh),grad(v))*dx +dot(dot(grad(uh),uh),v)*dx  -ph*div(v)*dx +q*div(uh)*dx  #dot for matrix vec multi

    return G

### FE spaces, data, BCs

In [ ]:
# Function spaces (mixed formulation)
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)
W = MixedFunctionSpace([V, Q])

# Data and boundary conditions
nu = 0.02
f = Constant((0.,0.))
x = SpatialCoordinate(mesh)

u_in = (4*x[1]*(1-x[1]), 0.)



bc_in = DirichletBC(W.sub(0),u_in , (1))

#bc_freeslip_x=DirichletBC(W.sub(1),Constant(0.), (32,42))
#bc_freeslip_y=DirichletBC(W.sub(1),Constant(0.), (31,33,41,43))

bc_wall=bc_freeslip_x= DirichletBC(W.sub(0),Constant((0.,0.)), (31,32,33,41,42,43))

bcs = (bc_in,bc_wall)


### Assembly of the residual

In [ ]:
wh = Function(W)
uh, ph = split(wh)
v, q = TestFunctions(W)
# NB:   wh (and also uh,ph) must be created before creating G.
#       If you re-define wh (or uh,ph), you MUST re-compute G,
#       since G is the evaluation of the residual on a specific wh.

# Notice that G needs to be a functional, not a bilinear form
G = nonlinear_problem_residual(uh, v, ph, q, nu, f)
print(len(G.arguments()))   # 1: only test function

u, p = TrialFunctions(W)
print(len((inner(u,v)*dx).arguments())) # 2: trial and test functions

### Actual solution

In [ ]:
param=( {'snes_type':'newtonls', # nonlinear solver types: https://petsc.org/release/manual/snes/#the-nonlinear-solvers
       'snes_monitor':'',        # activate logging options
       'snes_monitoring_lg_residualnorm':'', # print ||G|| at each iteration
       'snes_rtol':1e-6, 'snes_atol':1e-10, 'snes_stol':1e-14, 'snes_maxit':1000
       })
solve(G == 0, wh, bcs=bcs, solver_parameters=param)

# Equivalent:
#     nlvpb = NonlinearVariationalProblem(G, wh, bcs=bcs)
#     param = ...
#     solver = NonlinearVariationalSolver(nlvpb, solver_parameters=param)
#     solver.solve()

### Stream function problem

In [ ]:
Z = FunctionSpace(mesh, 'P', 1)
psi = TrialFunction(Z)
eta = TestFunction(Z)

a_stream = inner(grad(psi), grad(eta)) * dx
# omega = Dx(uh.sub(1),0) - Dx(uh.sub(0),1)   # by components (UFL syntax)
omega = rot(uh)                               # by pre-defined UFL command
L_stream = inner( omega , eta) * dx

# # On the mesh elbow3.msh
# bc_stream_1 = DirichletBC(Z, Constant(2.0/3.0), 3)
# bc_stream_2 = DirichletBC(Z, Constant(0.0), 4)

# On the mesh elbow3bis.msh
bc_stream_1 = DirichletBC(Z, Constant(2.0/3.0), (31,32,33))
bc_stream_2 = DirichletBC(Z, Constant(0.0), (41,42,43))

bcs_stream = (bc_stream_1, bc_stream_2)

psi_h = Function(Z)
solve(a_stream == L_stream, psi_h, bcs_stream)

### Post-processing

In [ ]:
# WARNING
# uh,ph were created as split(wh), by which uh,ph are proxies to access wh:
# this is NECESSARY for the residual G to have the suitable type for Firedrake's nonlinear solver.
# For post-processing, instead, we need a more "direct" access to the FE functions
# corresponding to the velocity and pressure component of the solution:
# this is achieved by wh.subfunctions
uh_out, ph_out = wh.subfunctions

fig, ax = plt.subplots()
col = tripcolor(ph_out, axes=ax)
plt.colorbar(col)
plt.title('pressure')

fig, ax = plt.subplots()
col = quiver(uh_out, axes=ax)
plt.colorbar(col)
plt.title('velocity')

col = tricontour(psi_h) # psi_h is already a Function: do not need to extract components
plt.colorbar(col)

# vtk output for Paraview
basename = 'lab07_elbow3_noslip_'
outfileU = File(my_io_path+"output/"+basename+"velocity.pvd")
outfileP = File(my_io_path+"output/"+basename+"pressure.pvd")
outfilePsi = File(my_io_path+"output/"+basename+"stream.pvd")
uh_out.rename("Velocity")   # this names will be used in Paraview
ph_out.rename("Pressure")
psi_h.rename("psi")
outfileU.write(uh_out)
outfileP.write(ph_out)
outfilePsi.write(psi_h)

In [ ]:
normal = FacetNormal(mesh)
drag = ...
print('Drag force on lower, downstream wall =', drag)

delta_p = (
    ...
)
print('Total pressure jump =', delta_p)

---
---
# Exercise 2 - homework

\begin{equation*}
\begin{cases}
\frac{\partial\boldsymbol{u}}{\partial t} + (\boldsymbol{u}\cdot\nabla)\boldsymbol{u} - \frac{1}{\rm Re}\Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega, t\in(0,T), \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega, t\in(0,T), \\
\boldsymbol{u} = 1\boldsymbol{i} & {\rm on} \ \Gamma_{\rm in}, t\in(0,T), \\
(\nu\nabla\boldsymbol{u} - pI)\boldsymbol{n} = \boldsymbol{0} & {\rm on} \ \Gamma_{\rm out}, t\in(0,T), \\
\boldsymbol{u}\cdot\boldsymbol{n} = 0, \quad (\nu\nabla\boldsymbol{u}-pI)\boldsymbol{n} \cdot \boldsymbol{t} = 0
& {\rm on} \ \Gamma_{\rm wall}, t\in(0,T),\\
\boldsymbol{u}=\boldsymbol{0} & {\rm on} \ \Gamma_{\rm cyl}, t\in(0,T),\\
\boldsymbol{u}=\boldsymbol{0} & {\rm in} \ \Omega, t=0.
\end{cases}
\end{equation*}

In [ ]:
# Import mesh: set the path correctly!
# See cell before Exercise 1 about current path.
mesh = Mesh(my_io_path+'meshes/cylinder-ns.msh')
fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend(loc='upper right')

### FE spaces, data, BCs

In [ ]:
...

### Variational problem - residual

In [ ]:
# inputs:   uh, ph  components of an actual Function (NOT trial functions)
#           v, q    components of a TestFunction
#           nu, f   viscosity and source function
#           dt      time step
#           uh_old  velocity at previous timestep
# output:   G       linear form (i.e. functional) representing the evaluation of
#                   the residual for a given solution (uh,ph), for any test function
def nonlinear_problem_residual(uh, v, ph, q, nu, f, dt, uh_old):
    G = ...
    return G

### Initial condition and settings

In [ ]:
uh_old = interpolate(Constant((0., 0.)), W.sub(0))

wh = Function(W)
uh, ph = split(wh)
# NB:   wh (and also uh,ph) must be created and properly set before assembling G.
#       If wh (or uh,ph) changes, you MUST re-assemble G,
#       since G is the evaluation of the residual on a specific wh.
uh_out, ph_out = wh.subfunctions
uh_out.assign(uh_old)       # must be done through a subfunction, not through a split component
v, q = TestFunctions(W)

t0 = 0
T = 10.
dt = 0.2
export_stride = 5     # export solution every other export_stride timesteps
time_index = 0

param = ...

basename = 'lab07_cylinder_'
outfileU = File(my_io_path+"output/"+basename+"velocity.pvd")
outfileP = File(my_io_path+"output/"+basename+"pressure.pvd")
outfilePsi = File(my_io_path+"output/"+basename+"stream.pvd")

In [ ]:
for t in np.arange(t0, T+0.1*dt, dt):
    print('\ntime t = ', t)
    time_index += 1

    ... assemble and solve ...

    if time_index % export_stride == 0:
        ... rename solution variables and write to pvd files ...

    uh_old.assign(uh_out)